<a href="https://colab.research.google.com/github/jiniwini/-Emotional-diary/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<strong>데이터 전처리</strong>

- 연속성 대화, 단발성 대화, 욕설 감지 데이터셋의 데이터 불러오기
- 필요없는 열 삭제 및 정제
  - 연속성 대화 : 청소년/청년, 사람문장1, 감정대분류 데이터만 사용
  - 욕설 감지 데이터셋 : 욕설에 해당하는 데이터만 사용
  - 단발성 대화 : 욕설 데이터를 [혐오]감정에 병합
- 통합하여 하나의 데이터로 만들기

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 단발성 대화 데이터셋 불러오기
short_raw = pd.read_excel('/content/drive/MyDrive/data/short_raw.xlsx')
short_raw = pd.DataFrame(short_raw)

In [ ]:
short_raw.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [ ]:
short_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  38594 non-null  object
 1   Emotion   38594 non-null  object
dtypes: object(2)
memory usage: 603.2+ KB


In [ ]:
short_raw.Emotion.value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [ ]:
# 연속성 대화 데이터셋(학습용) 불러오기
long_train = pd.read_excel('/content/drive/MyDrive/data/long_train.xlsx')
long_train = pd.DataFrame(long_train)
long_train.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [ ]:
long_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  51630 non-null  int64 
 1   연령          51630 non-null  object
 2   성별          51630 non-null  object
 3   상황키워드       51630 non-null  object
 4   신체질환        51630 non-null  object
 5   감정_대분류      51630 non-null  object
 6   감정_소분류      51630 non-null  object
 7   사람문장1       51630 non-null  object
 8   시스템문장1      51630 non-null  object
 9   사람문장2       51630 non-null  object
 10  시스템문장2      51630 non-null  object
 11  사람문장3       42695 non-null  object
 12  시스템문장3      42695 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.1+ MB


In [ ]:
long_train.감정_대분류.value_counts()

불안    9320
분노    9160
상처    9143
슬픔    9125
당황    8756
기쁨    6126
Name: 감정_대분류, dtype: int64

In [ ]:
long_train.columns

Index(['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3'],
      dtype='object')

In [ ]:
long_train.drop(['Unnamed: 0', '상황키워드', '성별', '신체질환', '감정_소분류', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3'],
                axis=1, inplace=True)
long_train.head()

,연령,감정_대분류,사람문장1
0,청년,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,청년,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,청년,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,청년,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,청년,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [ ]:
# 연속성 대화 데이터셋(테스트용) 불러오기
long_test = pd.read_excel('/content/drive/MyDrive/long_test.xlsx')
long_test = pd.DataFrame(long_test)
long_test.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.


In [ ]:
long_test.drop(['Unnamed: 0', '상황키워드', '성별', '신체질환', '감정_소분류', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3'],
                axis=1, inplace=True)
long_test.head()

,연령,감정_대분류,사람문장1
0,청년,불안,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1,청년,불안,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.
2,청년,불안,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.
3,청년,불안,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.
4,청년,불안,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.


In [ ]:
# 중년과 노년 데이터 삭제
idx2 = long_train[long_train['연령']=='중년'].index
idx2_1 = long_train[long_train['연령']=='노년'].index

In [ ]:
long_train.drop(idx2, inplace=True)
long_train.drop(idx2_1, inplace=True)
long_train.연령.value_counts()

청년     14832
청소년    10582
Name: 연령, dtype: int64

In [ ]:
long_train = long_train.rename(columns={"사람문장1":'Sentence',
                                        "감정_대분류":'Emotion'})

In [ ]:
long_train.head()

,연령,Emotion,Sentence
0,청년,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,청년,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,청년,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,청년,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,청년,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [ ]:
long_train.drop(['연령'], axis=1, inplace=True)

In [ ]:
long_train.head()

,Emotion,Sentence
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [ ]:
# short_raw 데이터셋의 [혐오]감정에 욕설 데이터셋 추가
# 욕설 데이터셋 불러오기
hate = pd.read_table('/content/drive/MyDrive/data/hate.txt', sep='/t')
hate.head()

<ipython-input-29-5a1739565494>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  hate = pd.read_table('/content/drive/MyDrive/Colab Notebooks/Final_project/data/hate.txt', sep='/t')


,Sentence
0,좌배 까는건 ㅇㅂ|1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ|0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아|1
3,세탁이라고 봐도 된다|0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ |1


In [ ]:
hate[hate['Sentence'].str.contains("t|0")]

,Sentence
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ|0
3,세탁이라고 봐도 된다|0
9,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌|0
10,ㅋㅋㅋㅋㅋ|0
11,ㄹㅇㅋㅋㅋ|0
...,...
5818,이쯤되니 전남대 전북대 상황이 궁금해지네 |0
5819,WIN10 업그레이드 했냐??|0
5820,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임|0
5822,너가 한 말 중에|0


In [ ]:
# 욕설 구분은 0과 1로 되어있음, 0에 해당하는 데이터 드롭
idx3 = hate[hate['Sentence'].str.contains("t|0")].index

In [ ]:
hate.drop(idx3, inplace=True)

In [ ]:
hate

,Sentence
0,좌배 까는건 ㅇㅂ|1
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아|1
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ |1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...
6,재앙이한건햇노|1
...,...
5811,응 그래 저능아새끼야 그렇게 생각하고 살렴. 저능아는 저능아대로살아야지 시발 불끄는...
5814,진짜 저 개성없는 머리는 왜 하는거냐|1
5815,오나1홀도 애1무 잘해주면 물 나온다 함 해보라ㅏ|1
5821,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같..."


In [ ]:
hate.reset_index(drop=True)

,Sentence
0,좌배 까는건 ㅇㅂ|1
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아|1
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ |1
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...
4,재앙이한건햇노|1
...,...
1938,응 그래 저능아새끼야 그렇게 생각하고 살렴. 저능아는 저능아대로살아야지 시발 불끄는...
1939,진짜 저 개성없는 머리는 왜 하는거냐|1
1940,오나1홀도 애1무 잘해주면 물 나온다 함 해보라ㅏ|1
1941,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같..."


In [ ]:
Sentence_1 = hate.Sentence

In [ ]:
hate['Sentence'] = Sentence_1.str.split('|').str[0]

In [ ]:
hate

,Sentence
0,좌배 까는건 ㅇㅂ
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...
6,재앙이한건햇노
...,...
5811,응 그래 저능아새끼야 그렇게 생각하고 살렴. 저능아는 저능아대로살아야지 시발 불끄는...
5814,진짜 저 개성없는 머리는 왜 하는거냐
5815,오나1홀도 애1무 잘해주면 물 나온다 함 해보라ㅏ
5821,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같..."


In [ ]:
hate['Emotion'] = Sentence_1.str.split('|').str[1]
hate.head()

,Sentence,Emotion
0,좌배 까는건 ㅇㅂ,1
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
6,재앙이한건햇노,1


In [ ]:
hate.reset_index(drop=True)

,Sentence,Emotion
0,좌배 까는건 ㅇㅂ,1
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
4,재앙이한건햇노,1
...,...,...
1938,응 그래 저능아새끼야 그렇게 생각하고 살렴. 저능아는 저능아대로살아야지 시발 불끄는...,1
1939,진짜 저 개성없는 머리는 왜 하는거냐,1
1940,오나1홀도 애1무 잘해주면 물 나온다 함 해보라ㅏ,1
1941,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1


In [ ]:
# label==1을 혐오로 바꿔주기
hate.loc[hate['Emotion']== '1', 'Emotion'] = '혐오'

In [ ]:
hate

,Sentence,Emotion
0,좌배 까는건 ㅇㅂ,혐오
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,혐오
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,혐오
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,혐오
6,재앙이한건햇노,혐오
...,...,...
5811,응 그래 저능아새끼야 그렇게 생각하고 살렴. 저능아는 저능아대로살아야지 시발 불끄는...,혐오
5814,진짜 저 개성없는 머리는 왜 하는거냐,혐오
5815,오나1홀도 애1무 잘해주면 물 나온다 함 해보라ㅏ,혐오
5821,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",혐오


In [ ]:
# 욕설 데이터셋 short_raw에 병합
short_raw = pd.concat([short_raw, hate], axis=0)

In [ ]:
# short_raw train/test 데이터셋 분리
from sklearn.model_selection import train_test_split
short_train, short_test = train_test_split(short_raw, test_size=0.2, random_state=1004)

In [ ]:
short_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32429 entries, 9096 to 20511
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  32429 non-null  object
 1   Emotion   32429 non-null  object
dtypes: object(2)
memory usage: 760.1+ KB


In [ ]:
short_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8108 entries, 1886 to 13366
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  8108 non-null   object
 1   Emotion   8108 non-null   object
dtypes: object(2)
memory usage: 190.0+ KB


In [ ]:
short_train.head()

,Sentence,Emotion
9096,얘네는 댓글만 보면 중국가수네,놀람
37364,니들이 1~2년 쓰다가 고장나는건 폰을 개떡같이 쓰니까 그런거야 펴엉신들아ㅉㅉ앱등이 수준,혐오
29749,항상 감사합니다 용규선수ㅠㅠㅠㅠ,행복
36360,박근혜와 친박 개누리들이 버틴다면 세자릿수도 가능할듯,혐오
2748,질싸?,혐오


In [ ]:
short_train = short_train.reset_index(drop=True)

In [ ]:
short_train.head()

,Sentence,Emotion
0,얘네는 댓글만 보면 중국가수네,놀람
1,니들이 1~2년 쓰다가 고장나는건 폰을 개떡같이 쓰니까 그런거야 펴엉신들아ㅉㅉ앱등이 수준,혐오
2,항상 감사합니다 용규선수ㅠㅠㅠㅠ,행복
3,박근혜와 친박 개누리들이 버틴다면 세자릿수도 가능할듯,혐오
4,질싸?,혐오


In [ ]:
long_train

,Emotion,Sentence
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51613,당황,부모님이 내가 밖에 나가려고만 하면 간섭을 하셔. 속상해.
51614,불안,부모님은 나에게 관심도 없어. 내가 무슨 말을 하든 다 잊어버리는 것 같아서 짜증 나.
51615,슬픔,내가 지금 할 수 있는 건 아무것도 없어. 아내는 계속 아르바이트라도 알아보라고 해.
51616,당황,결혼 후 독립에 대한 양가 부모님의 생각이 달라. 어떻게 하면 좋을지 모르겠어.


In [ ]:
# long_train과 short_train 병합
total_train = pd.concat([short_train, long_train], axis=0)

In [ ]:
long_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25414 entries, 0 to 51617
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Emotion   25414 non-null  object
 1   Sentence  25414 non-null  object
dtypes: object(2)
memory usage: 595.6+ KB


In [ ]:
long_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3010 entries, 0 to 6640
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Emotion   3010 non-null   object
 1   Sentence  3010 non-null   object
dtypes: object(2)
memory usage: 70.5+ KB


In [ ]:
total_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57843 entries, 0 to 51617
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  57843 non-null  object
 1   Emotion   57843 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
short_test = short_test.reset_index(drop=True)
short_test.head()

,Sentence,Emotion
0,지디게이 죽여 없애야됨,혐오
1,친일파들을 죽여라~,혐오
2,착각들 하는데 얼굴이 큰거랑 머리가 큰거는 다른뎅,중립
3,개 씝 극우 쪽빠리 수장 새끼,혐오
4,삼성이 돈을 조금더 벌려고 싸구려 중국산 밧데리 장착했다가 개망신 당하는구나!,분노


In [ ]:
long_test.연령.value_counts()

노년     2053
중년     1578
청년     1573
청소년    1437
Name: 연령, dtype: int64

In [ ]:
long_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3010 entries, 0 to 6640
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Emotion   3010 non-null   object
 1   Sentence  3010 non-null   object
dtypes: object(2)
memory usage: 70.5+ KB


In [ ]:
# 중년과 노년 데이터 삭제
idx1 = long_test[long_test['연령']=='중년'].index
idx1_1 = long_test[long_test['연령']=='노년'].index

In [ ]:
long_test.drop(idx1, inplace=True)
long_test.drop(idx1_1, inplace=True)
long_test.연령.value_counts()

청년     1573
청소년    1437
Name: 연령, dtype: int64

In [ ]:
long_test = long_test.rename(columns={"사람문장1":'Sentence',
                                        "감정_대분류":'Emotion'})

In [ ]:
# long_test와 short_test 병합
long_test.drop(['연령'], axis=1, inplace=True)
total_test = pd.concat([short_test, long_test], axis=0)
total_test.Emotion.value_counts()

분노    1728
슬픔    1568
혐오    1450
행복    1215
놀람    1182
공포    1095
중립     960
당황     521
기쁨     517
상처     451
불안     431
Name: Emotion, dtype: int64

In [ ]:
total_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11118 entries, 0 to 6640
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  11118 non-null  object
 1   Emotion   11118 non-null  object
dtypes: object(2)
memory usage: 260.6+ KB


In [ ]:
total_test

,Sentence,Emotion
0,지디게이 죽여 없애야됨,혐오
1,친일파들을 죽여라~,혐오
2,착각들 하는데 얼굴이 큰거랑 머리가 큰거는 다른뎅,중립
3,개 씝 극우 쪽빠리 수장 새끼,혐오
4,삼성이 돈을 조금더 벌려고 싸구려 중국산 밧데리 장착했다가 개망신 당하는구나!,분노
...,...,...
6636,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,당황
6637,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,당황
6638,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,당황
6639,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,당황


In [ ]:
total_train

,Sentence,Emotion
0,얘네는 댓글만 보면 중국가수네,놀람
1,니들이 1~2년 쓰다가 고장나는건 폰을 개떡같이 쓰니까 그런거야 펴엉신들아ㅉㅉ앱등이 수준,혐오
2,항상 감사합니다 용규선수ㅠㅠㅠㅠ,행복
3,박근혜와 친박 개누리들이 버틴다면 세자릿수도 가능할듯,혐오
4,질싸?,혐오
...,...,...
51613,부모님이 내가 밖에 나가려고만 하면 간섭을 하셔. 속상해.,당황
51614,부모님은 나에게 관심도 없어. 내가 무슨 말을 하든 다 잊어버리는 것 같아서 짜증 나.,불안
51615,내가 지금 할 수 있는 건 아무것도 없어. 아내는 계속 아르바이트라도 알아보라고 해.,슬픔
51616,결혼 후 독립에 대한 양가 부모님의 생각이 달라. 어떻게 하면 좋을지 모르겠어.,당황


In [ ]:
total_train.Emotion.value_counts()

분노    8931
슬픔    8724
혐오    5922
행복    4822
놀람    4716
불안    4598
상처    4395
공포    4373
당황    4248
중립    3870
기쁨    3244
Name: Emotion, dtype: int64

In [ ]:
long_train.Emotion.value_counts()

불안    4598
슬픔    4529
분노    4400
상처    4395
당황    4248
기쁨    3244
Name: Emotion, dtype: int64

In [ ]:
short_train.Emotion.value_counts()

혐오    5922
행복    4822
놀람    4716
분노    4531
공포    4373
슬픔    4195
중립    3870
Name: Emotion, dtype: int64

In [ ]:
total_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57843 entries, 0 to 51617
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  57843 non-null  object
 1   Emotion   57843 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
total_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11118 entries, 0 to 6640
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  11118 non-null  object
 1   Emotion   11118 non-null  object
dtypes: object(2)
memory usage: 260.6+ KB


In [ ]:
# # 데이터 저장
total_test.to_csv('/content/drive/MyDrive/data/total_test_real.csv')
total_train.to_csv('/content/drive/MyDrive/data/total_train_real.csv')

In [ ]:
total_test.Emotion.value_counts()

분노    1728
슬픔    1568
혐오    1450
행복    1215
놀람    1182
공포    1095
중립     960
당황     521
기쁨     517
상처     451
불안     431
Name: Emotion, dtype: int64